In [2]:
include("iLQR.jl")
using iLQR
using Base.Test
using BenchmarkTools

In [30]:
n,m = 3,2
A = [0 1 0; 0 0 1; 0 0 0]
B = [0 0; 0 1; 1 1]
x = [1;2;3]
u = [4;5]

Qf = diagm([10, 20, 30])
R = eye(m)*2
Q = diagm([3, 2, 1])

3×3 Array{Int64,2}:
 3  0  0
 0  2  0
 0  0  1

In [19]:
S = copy(Qf)
S̄ = A'S*A - A'S*B*inv(B'S*B + R)B'S*A + Q

L = chol(S)'
@test L*L' ≈ S
Lr = chol(R)'
Lq = chol(Q)'

G = L'B*inv(Lr')
W̃,Ṽ = qr(G)
D̃ = Ṽ'Ṽ
W,D = my_normalize(W̃,D̃)
@test norm(W[:,1]) == 1
@test W*D ≈ W̃*D̃

G2 = chol(I+D)'
L̄ = A'L*W*G2*W'


3×3 Array{Float64,2}:
 0.0   0.0       0.0   
 0.0   0.0       0.0   
 0.0  15.4515  -16.7705

In [20]:
function my_normalize(W,D)
    n,m = size(W)
    norms = zeros(m)
    W2 = similar(W)
    for i = 1:m
        norms[i] = norm(W[:,i])
        W2[:,i] = W[:,i]/norms[i]
    end
    D2 = diagm(norms.^2)*D
    return W2,D2
end


my_normalize (generic function with 1 method)

In [22]:
@test S̄ ≈ A'L*inv(eye(n)+G*G')L'A + Q
W̃,Ṽ = qr(G)
W,R = my_normalize(W̃,Ṽ*Ṽ')
@test S̄ ≈ A'L*inv(eye(n)+W*R*W')L'A + Q
D,E = eig(R)
D = diagm(D)
@test S̄ ≈ A'L*inv(eye(n)+W*E*D*E'*W')L'A + Q
eig(G*G')[2]
Gaug = [G zeros(3)]
D,V = eig(G*G')
D = diagm(D)
Daug,Vaug = eig(Gaug)
Daug = diagm(Daug)
@test Gaug*Gaug' ≈ G*G'
V*D*V' ≈ G*G'
Vaug*Daug*Vaug'
V*V'

3×3 Array{Float64,2}:
 1.0  0.0          0.0        
 0.0  1.0          5.55112e-17
 0.0  5.55112e-17  1.0        

In [26]:
# Method 2
M = [inv(L)' B*inv(Lr')]
@show size(M)
S̄ ≈ A'inv(M*M')*A + Q
G = [A'/M' Lq]
@test S̄ ≈ G*G'
Qg,R = qr(G)
@test S̄ ≈ Qg*R*R'*Qg'
D,V = eig(R*R')
D = diagm(D)
@test V*D*V' ≈ R*R'
@test S̄ ≈ Q*V*sqrt.(D)*sqrt.(D)*V'*Qg'
L̄ = Qg*V*sqrt.(D)
@test S̄ ≈ L̄*L̄'

size(M) = (3, 5)


Test Passed

In [81]:
# Method 3 (Manchester Way)
U = chol(S)
Uq = chol(Q)
Ur = chol(R)
_,Wxx = qr(-[U*A; Uq])
@test chol(A'S*A + Q) ≈ Wxx
Wuu = qrfact(-[U*B; Ur])
@test chol(R+B'S*B) ≈ Wuu[:R]
Qxu = A'U'U*B
# S̄ ≈ Wxx'Wxx - Qxu*(Wuu\(Wuu'\Qxu'))

AmB = LinAlg.Cholesky(copy(Wxx),'U')
Ub = Wuu[:R]'\Qxu'
# U2 = (Qxu/Wuu)
for i = 1:m
    LinAlg.lowrankdowndate!(AmB,Ub[i,:])
end
U2 = AmB[:U]
@test S̄ ≈ U2'U2

Error During Test
  Test threw an exception of type ArgumentError
  Expression: chol((A' * S) * A + Q) ≈ Wxx
  ArgumentError: matrix is not symmetric/Hermitian. This error can be avoided by calling chol(Hermitian(A)) which will ignore either the upper or lower triangle of the matrix.
  Stacktrace:
   [1] chol(::Array{Float64,2}) at ./linalg/cholesky.jl:185
   [2] include_string(::String, ::String) at ./loading.jl:522
   [3] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/bjack205/.julia/v0.6/IJulia/src/execute_request.jl:158
   [4] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/bjack205/.julia/v0.6/Compat/src/Compat.jl:378
   [5] eventloop(::ZMQ.Socket) at /home/bjack205/.julia/v0.6/IJulia/src/eventloop.jl:8
   [6] (::IJulia.##14#17)() at ./task.jl:335


LoadError: [91mThere was an error during testing[39m

In [89]:
qrfact(-[U*B; Ur; eye(2)*1])[:R] ≈ qrfact(-[Wuu[:R]; eye(2)*1])[:R]
@btime qrfact(-[U*B; Ur; eye(2)*1])[:R]
@btime Wuu = qrfact(-[U*B; Ur]); qrfact(-[Wuu[:R]; chol(eye(2)*1)])[:R]
@btime Wuu = qrfact(-[U*B; Ur]); qrfact(-[Wuu[:R]; sqrt.(eye(2)*1)])[:R]

LoadError: [91mArgumentError: number of columns of each array must match (got (4, 4, 2))[39m

In [29]:
A = -[U*B; Ur]
tmp = qrfact!(A)
tmp

Base.LinAlg.QRCompactWY{Float64,Array{Float64,2}} with factors Q and R:
[0.0 1.99347e-16 … -0.25 3.01248e-17; 0.0 -0.915258 … 0.27134 -0.28943; … ; -0.25 0.27134 … 0.899058 0.0410044; 0.0 -0.28943 … 0.0410044 0.956262]
[5.65685 5.3033; 0.0 4.88621]

In [121]:
n, m = 30,20
A = rand(n,n)
B = rand(n,m)

Qf = diagm(rand(n)*10)
R = eye(m)*2*rand()
Q = diagm(rand(n)*3)

30×30 Array{Float64,2}:
 0.417764  0.0      0.0      0.0      …  0.0      0.0      0.0       0.0    
 0.0       1.65583  0.0      0.0         0.0      0.0      0.0       0.0    
 0.0       0.0      2.43595  0.0         0.0      0.0      0.0       0.0    
 0.0       0.0      0.0      1.31952     0.0      0.0      0.0       0.0    
 0.0       0.0      0.0      0.0         0.0      0.0      0.0       0.0    
 0.0       0.0      0.0      0.0      …  0.0      0.0      0.0       0.0    
 0.0       0.0      0.0      0.0         0.0      0.0      0.0       0.0    
 0.0       0.0      0.0      0.0         0.0      0.0      0.0       0.0    
 0.0       0.0      0.0      0.0         0.0      0.0      0.0       0.0    
 0.0       0.0      0.0      0.0         0.0      0.0      0.0       0.0    
 0.0       0.0      0.0      0.0      …  0.0      0.0      0.0       0.0    
 0.0       0.0      0.0      0.0         0.0      0.0      0.0       0.0    
 0.0       0.0      0.0      0.0         0.0      0.

In [122]:
S = copy(Qf)
function updateFull(S)
    A'S*A - A'S*B*inv(B'S*B + R)B'S*A + Q
end
S̄ = updateFull(S)


L = chol(S)'
@test L*L' ≈ S
Lr = chol(R)'
iLr = inv(Lr)
Lq = chol(Q)'

U = chol(S)
Ur = chol(R)
Uq = chol(Q)


30×30 UpperTriangular{Float64,Array{Float64,2}}:
 0.646347  0.0      0.0      0.0     …  0.0      0.0      0.0       0.0   
  ⋅        1.28679  0.0      0.0        0.0      0.0      0.0       0.0   
  ⋅         ⋅       1.56075  0.0        0.0      0.0      0.0       0.0   
  ⋅         ⋅        ⋅       1.1487     0.0      0.0      0.0       0.0   
  ⋅         ⋅        ⋅        ⋅         0.0      0.0      0.0       0.0   
  ⋅         ⋅        ⋅        ⋅      …  0.0      0.0      0.0       0.0   
  ⋅         ⋅        ⋅        ⋅         0.0      0.0      0.0       0.0   
  ⋅         ⋅        ⋅        ⋅         0.0      0.0      0.0       0.0   
  ⋅         ⋅        ⋅        ⋅         0.0      0.0      0.0       0.0   
  ⋅         ⋅        ⋅        ⋅         0.0      0.0      0.0       0.0   
  ⋅         ⋅        ⋅        ⋅      …  0.0      0.0      0.0       0.0   
  ⋅         ⋅        ⋅        ⋅         0.0      0.0      0.0       0.0   
  ⋅         ⋅        ⋅        ⋅         0.0      0.

In [136]:
function chol_minus(A,B)
    AmB = LinAlg.Cholesky(copy(A),'U')
    for i = 1:size(B,1)
        LinAlg.lowrankdowndate!(AmB,B[i,:])
    end
    U = AmB[:U]
end

function update(L)
    M = [inv(L)' B*inv(Lr')]
    G = [A'/M' Lq]
    Q,R = qr(G)
    D,V = eig(R*R')
    D = diagm(D)
    L̄ = Q*V*sqrt.(D)
end
L̄ = update(L)
@test S̄ ≈ L̄*L̄'

function update2(L)
    G = L'*B/(Lr')
    d1, W1 = eig(G*G')
    D1 = diagm(d1)
    P = [A'*L*W1*((eye(n) + D1)^(-1/2))*W1' Lq]

    W1, V = qr(P)
    d2, W2 = eig(V*V')
    D2 = diagm(d2)
    L_ = W1*W2*D2^(1/2)
end
L̄2 = update2(L)
@test S̄ ≈ L̄2*L̄2'

μ = 2.5
function update3(U)
    Wxx = qrfact!([U*A; Uq])
    Wuu = qrfact!([U*B; Ur; chol(eye(size(B,2))*μ)])
    
    Qxu = A'U'U*B
    Ub = Wuu[:R]'\Qxu'
    L_ = chol_minus(Wxx[:R],Ub)
end
L̄3 = update3(U)
# @test S̄ ≈ L̄3'L̄3

function update3_(U,μ)
    Wxx = qrfact!([U*A; Uq])
    if μ > 0
        Wuu = qrfact!([U*B; Ur; chol(eye(size(B,2))*μ)])
    else
        Wuu = qrfact!([U*B; Ur])
    end
    
    Qxu = A'U'U*B
    Ub = Wuu[:R]'\Qxu'
    L_ = chol_minus(Wxx[:R],Ub)
end
update3_(U)

function update4(U)
    Wxx = qrfact!([U*A; Uq])
    Wuu = qrfact!([qrfact!([U*B; Ur])[:R]; chol(eye(size(B,2))*μ)])
    
    Qxu = A'U'U*B
    K = Wuu[:R]\(Wuu[:R]'\Qxu')
    L_ = chol_minus(Wxx[:R],Wuu[:R]'\Qxu')
   
end
L̄4 = update4(U)
# @test S̄ ≈ L̄4'L̄4
L̄4 ≈ L̄3


true

In [226]:
@btime update(L)
@btime update2(L)
@btime update4(U)
@btime updateFull(S)

  183.190 μs (238 allocations: 140.84 KiB)
  308.696 μs (124 allocations: 76.09 KiB)
  62.046 μs (78 allocations: 16.61 KiB)
  6.815 μs (23 allocations: 14.00 KiB)


12×12 Array{Float64,2}:
  7.61656    1.32156   3.16768   …   2.89457     3.92608    2.03177  
  1.32156    4.77262   2.6708        0.729421    2.50999   -0.249376 
  3.16768    2.6708    3.30358       1.77091     2.80404    0.856622 
  4.58941    0.172512  1.39235       1.11983     2.61231    1.01561  
  2.85224    0.933054  1.43291       2.03348     3.50394   -0.349679 
 -0.26304    2.08818   0.933434  …   0.19373     1.04955   -0.357903 
  5.79737    1.34365   3.01626       4.26407     4.56977    1.08284  
  0.872674   1.49311   0.701604     -0.406199    1.33542   -0.806062 
  1.23312    0.170059  0.157723      0.0526358   1.31634   -0.0463986
  2.89457    0.729421  1.77091       3.46715     2.40825    0.928833 
  3.92608    2.50999   2.80404   …   2.40825     5.31023   -0.044313 
  2.03177   -0.249376  0.856622      0.928833   -0.044313   3.73521  

In [140]:
# @btime update3(U);
μ = 1.2
@btime update3_(U,μ);
μ = 0.
@btime update3_(U,μ);

  271.975 μs (100 allocations: 125.50 KiB)
  310.639 μs (89 allocations: 112.44 KiB)


In [220]:
@btime update4(U);

  701.576 μs (231 allocations: 258.23 KiB)


# Pendulum Test

In [2]:
## Double pendulum
urdf_dp = "urdf/doublependulum.urdf"
dp = iLQR.Model(urdf_dp)

# initial and goal states
x0 = [0.;0.;0.;0.]
xf = [pi;0.;0.;0.]

# costs
Q = 0.0001*eye(4)
Qf = 250.0*eye(4)
R = 0.0001*eye(2)

# simulation
tf = 5.0
dt = 0.1

obj = iLQR.Objective(Q,R,Qf,tf,x0,xf)
solver = iLQR.Solver(dp,obj,dt=dt);

In [291]:
# Initialization
X = zeros(n,N)
U = 10.0*ones(solver.model.m,solver.N)
K = zeros(m,n,N-1)
d = zeros(m,N-1)
rollout!(solver, X, U)


N = solver.N
n = solver.model.n
m = solver.model.m
Q = solver.obj.Q
R = solver.obj.R
xf = solver.obj.xf
Qf = solver.obj.Qf

S = Qf
s = Qf*(X[:,N] - xf)
v1 = 0.
v2 = 0.

mu = 0.

# Backwards Pass
k = 1
lx = Q*(X[:,k] - xf)
lu = R*(U[:,k])
lxx = Q
luu = R
fx, fu = solver.F(X[:,k],U[:,k])
Qx = lx + fx'*s
Qu = lu + fu'*s
Qxx = lxx + fx'*S*fx
Quu = luu + fu'*(S + mu*eye(n))*fu
Qux = fu'*(S + mu*eye(n))*fx

# regularization
if any(eigvals(Quu).<0.)
    mu = mu + 1.0;
    k = N-1;
    println("regularized")
end

K[:,:,k] = Quu\Qux
d[:,k] = Quu\Qu
s = (Qx' - Qu'*K[:,:,k] + d[:,k]'*Quu*K[:,:,k] - d[:,k]'*Qux)'
S = Qxx + K[:,:,k]'*Quu*K[:,:,k] - K[:,:,k]'*Qux - Qux'*K[:,:,k]

4×4 Array{Float64,2}:
 249.425      -2.30009   12.4794    -0.254254
  -2.30009   258.462     -0.316831  13.1012  
  12.4794     -0.316831   0.900305   0.118705
  -0.254254   13.1012     0.118705   0.7373  

In [3]:
function chol_minus(A,B)
    AmB = LinAlg.Cholesky(copy(A),'U')
    for i = 1:size(B,1)
        LinAlg.lowrankdowndate!(AmB,B[i,:])
    end
    U = AmB[:U]
end

chol_minus (generic function with 1 method)

In [6]:
# Initialization
m,n = solver.model.m, solver.model.n
N = solver.N
X = zeros(solver.model.n,solver.N)
U = 10.0*ones(solver.model.m,solver.N)
K = zeros(m,n,N-1)
d = zeros(m,N-1)
rollout!(solver, X, U)


N = solver.N
n = solver.model.n
m = solver.model.m
Q = solver.obj.Q
R = solver.obj.R
xf = solver.obj.xf
Qf = solver.obj.Qf

S0 = Qf
S = Qf
s = Qf*(X[:,N] - xf)
v1 = 0.
v2 = 0.

mu = 0.

# Pre-computation
Uq = chol(Q)
Ur = chol(R)
Su = chol(S)

# Backwards Pass
k = 1
lx = Q*(X[:,k] - xf)
lu = R*(U[:,k])
lxx = Q
luu = R
fx, fu = solver.F(X[:,k],U[:,k])
Qx = lx + fx'*s
Qu = lu + fu'*s
Qxx = lxx + fx'*S*fx
Quu = luu + fu'*(S + mu*eye(n))*fu
Qux = fu'*(S + mu*eye(n))*fx

# regularization
if any(eigvals(Quu).<0.)
    mu = mu + 1.0;
    k = N-1;
    println("regularized")
end

K[:,:,k] = Quu\Qux
d[:,k] = Quu\Qu
s = (Qx' - Qu'*K[:,:,k] + d[:,k]'*Quu*K[:,:,k] - d[:,k]'*Qux)'
S = Qxx + K[:,:,k]'*Quu*K[:,:,k] - K[:,:,k]'*Qux - Qux'*K[:,:,k]

Wxx = qrfact([Su*fx; Uq])
Wuu = qrfact([Su*fu; Ur])
@test Wxx[:R]'Wxx[:R] ≈ Qxx
@test Wuu[:R]'Wuu[:R] ≈ Quu
@test Su'Su ≈ S0

Qxu = fx'*(Su'Su)*fu
@test Qxu ≈ Qux'

Knew = Wuu[:R]\(Wuu[:R]'\Qxu')
dnew = Wuu[:R]\(Wuu[:R]'\Qu)
@test K[:,:,k] ≈ Knew
@test dnew ≈ d[:,k]


s_ = Qx' - (Wuu[:R]'\Qu)'*(Wuu[:R]'\Qxu')
@test s' ≈ s_

Su_ = chol_minus(Wxx[:R],Wuu[:R]'\Qxu')
@test Su_'Su_ ≈ S



Test Passed

Test Failed
  Expression: Ksr[:, :, :] ≈ Kf[:, :, :]
   Evaluated: [-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -8.642 -10.1267 22.9862 12.9495]

[-22.1799 -7.87738 45.8446 23.024; -

LoadError: [91mThere was an error during testing[39m

In [30]:
function backwards_sqrt(solver::Solver,X::Array{Float64,2},U::Array{Float64,2},K::Array{Float64,3},d::Array{Float64,2})
    N = solver.N
    n = solver.model.n
    m = solver.model.m
    Q = solver.obj.Q
    R = solver.obj.R
    xf = solver.obj.xf
    Qf = solver.obj.Qf
    
    Uq = chol(Q)
    Ur = chol(R)
    Su = chol(Qf)

#     S = Qf
    s = Qf*(X[:,N] - xf)
    v1 = 0.
    v2 = 0.

    mu = 0.
    k = N-1
    while k >= 1
        lx = Q*(X[:,k] - xf)
        lu = R*(U[:,k])
        lxx = Q
        luu = R
        fx, fu = solver.F(X[:,k],U[:,k])
        Qx = lx + fx'*s
        Qu = lu + fu'*s
        
        Wxx = qrfact([Su*fx; Uq])
        Wuu = qrfact([Su*fu; Ur])
        Qxu = fx'*(Su'Su)*fu
#         Qxx = lxx + fx'*S*fx
#         Quu = luu + fu'*(S + mu*eye(n))*fu
#         Qux = fu'*(S + mu*eye(n))*fx

        # regularization
#         if any(eigvals(Quu).<0.)
#             mu = mu + 1.0;
#             k = N-1;
#             println("regularized")
#         end
        
        K[:,:,k] = Wuu[:R]\(Wuu[:R]'\Qxu')
        d[:,k] = Wuu[:R]\(Wuu[:R]'\Qu)
#         K[:,:,k] = Quu\Qux
#         d[:,k] = Quu\Qu
        
        s = (Qx' - (Wuu[:R]'\Qu)'*(Wuu[:R]'\Qxu'))'
        Su = chol_minus(Wxx[:R],Wuu[:R]'\Qxu')
#         s = (Qx' - Qu'*K[:,:,k] + d[:,k]'*Quu*K[:,:,k] - d[:,k]'*Qux)'
#         S = Qxx + K[:,:,k]'*Quu*K[:,:,k] - K[:,:,k]'*Qux - Qux'*K[:,:,k]
        

        # terms for line search
        v1 += float(d[:,k]'*Qu)[1]
        v2 += float(d[:,k]'*Wuu[:R]'Wuu[:R]*d[:,k])

        k = k - 1;
    end
    return K, d, v1, v2
    
end


backwards_sqrt (generic function with 1 method)

In [36]:
function backwards_full(solver::Solver,X::Array{Float64,2},U::Array{Float64,2},K::Array{Float64,3},d::Array{Float64,2})
    N = solver.N
    n = solver.model.n
    m = solver.model.m
    Q = solver.obj.Q
    R = solver.obj.R
    xf = solver.obj.xf
    Qf = solver.obj.Qf

    S = Qf
    s = Qf*(X[:,N] - xf)
    v1 = 0.
    v2 = 0.

    mu = 0.
    k = N-1
    while k >= 1
        lx = Q*(X[:,k] - xf)
        lu = R*(U[:,k])
        lxx = Q
        luu = R
        fx, fu = solver.F(X[:,k],U[:,k])
        Qx = lx + fx'*s
        Qu = lu + fu'*s
        Qxx = lxx + fx'*S*fx
        Quu = luu + fu'*(S + mu*eye(n))*fu
        Qux = fu'*(S + mu*eye(n))*fx

        # regularization
        if any(eigvals(Quu).<0.)
            mu = mu + 1.0;
            k = N-1;
            println("regularized")
        end

        K[:,:,k] = Quu\Qux
        d[:,k] = Quu\Qu
        s = (Qx' - Qu'*K[:,:,k] + d[:,k]'*Quu*K[:,:,k] - d[:,k]'*Qux)'
        S = Qxx + K[:,:,k]'*Quu*K[:,:,k] - K[:,:,k]'*Qux - Qux'*K[:,:,k]

        # terms for line search
        v1 += float(d[:,k]'*Qu)[1]
        v2 += float(d[:,k]'*Quu*d[:,k])

        k = k - 1;
    end
    return K, d, v1, v2
    
end


backwards_full (generic function with 1 method)

In [47]:
m,n = solver.model.m, solver.model.n
N = solver.N
X = zeros(solver.model.n,solver.N)
U = 10.0*ones(solver.model.m,solver.N)
K = zeros(m,n,N-1)
d = zeros(m,N-1)

rollout!(solver, X, U)
@time Ksr,dsr,v1sr,v2sr = backwards_sqrt(solver,X,U,K,d);

X = zeros(solver.model.n,solver.N)
U = 10.0*ones(solver.model.m,solver.N)
K = zeros(m,n,N-1)
d = zeros(m,N-1)

rollout!(solver, X, U)
@time Kf,df,v1f,v2f = backwards_full(solver,X,U,K,d);
@test Ksr[:,:,:] ≈ Kf[:,:,:]
@test dsr ≈ df
@test v1sr ≈ v1f
@test v2sr ≈ v2f

  0.134365 seconds (642.57 k allocations: 65.575 MiB, 11.90% gc time)
  0.178857 seconds (643.68 k allocations: 65.896 MiB, 7.81% gc time)
Test Failed
  Expression: Ksr[:, :, :] ≈ Kf[:, :, :]
   Evaluated: [0.641321 1.29903 1.21707 0.394005; -2.50138 -3.37577 1.36256 1.21122]

[0.419465 1.61156 0.85649 0.256942; -1.40048 -3.17275 2.25445 1.6727]

[-0.395737 0.55023 0.856064 0.200009; 0.75285 0.213917 2.38347 1.98797]

[-1.05975 -0.167539 1.04002 0.209704; 2.91601 2.56987 1.91088 2.06177]

[-1.35318 -0.495977 1.1755 0.189102; 4.59559 3.42747 1.29498 2.00526]

[-1.35134 -0.72344 1.1593 0.0755855; 5.67739 3.63514 0.84006 2.00028]

[-1.24249 -0.849501 0.978048 -0.113022; 6.30155 3.57472 0.71158 2.18745]

[-1.23259 -0.800393 0.698311 -0.286642; 6.76384 3.33769 1.01529 2.62408]

[-1.43453 -0.655237 0.469383 -0.336836; 7.40901 3.11499 1.80107 3.28362]

[-1.77336 -0.545641 0.490806 -0.189607; 8.49512 3.38643 3.01358 4.09557]

[-1.98698 -0.411328 0.954223 0.19765; 10.0595 4.66246 4.44834 4.9774

LoadError: [91mThere was an error during testing[39m

# PSD Matrices

In [298]:
function makePSD_current(A)
    n = size(A,1)
    mu = 0
    for i = 1:100
        if any(eigvals(A).<0.)
            mu += 1
            A += mu*eye(n)
        else
            break;
        end
    end
    return A
end

function makePSD_eig(A)
    δ = 1e-3
    λ,Q = eig(A)
    λ[λ .< δ] = δ
    Q*diagm(λ)Q'
end

function makePSD_cur_mod(A,δ=1.)
    n = size(A,1)
    mu = 0
    for i = 1:100
        if ~isposdef(A)
            mu += δ
            A += mu*eye(n)
        else
            break;
        end
    end
    return A
end

function makePSD_33(A)
    n = size(A,1)
    I = eye(n)

    β = 1e-3
    min_diag = minimum(diag(A))
    if min_diag > 0
        τ = 0
    else
        τ = -min_diag + β
    end
    
    for k = 1:100
        v = eigvals(A)
        if isposdef(A+τ*I)
#             println("iter: $k")
            return A+τ*I
        else
            τ = max(2*τ,β)
        end
    end
    return A
end

function makePSD_33_mod(A)
    n = size(A,1)
    I = eye(n)
    mu = 0
    
    β = 1e-2
    min_diag = minimum(diag(A))
    if min_diag > 0
        mu = 0
        δ = 1
    else
        mu = -min_diag + β
        δ = -min_diag
    end
    A += mu*I
    min_diag = abs(min_diag)
    
    for k = 1:100
        if ~isposdef(A)
            mu += δ
            A += mu*I
        else
#             println("iter: $k")
            break;
        end
    end
    return A
end

function makePSD_chol(A::Array)
    n = size(A,1)
    d = zeros(n)
    C = zeros(A)
    L = eye(A)
    β = 1e-3
    δ = 1e-3
    for j = 1:n
        t = 0
        for s = 1:j-1
            t += d[s]*L[j,s]^2
        end
        C[j,j] = A[j,j] - t
#         d[j] = cjj
        θ = maximum(C[j:n,j])
        d[j] = max(abs(C[j,j]),(θ\β)^2,δ)
        
        for i = j+1:n
            t = 0
            for s = 1:j-1
                t += d[s]*L[i,s]*L[j,s]
            end
            C[i,j] = A[i,j] - t
            L[i,j] = C[i,j] / d[j]
        end
    end
    D = diagm(d)
    return L,D
end
@time L,D = makePSD_chol(A)
@time chol(A)
isposdef(L*sqrt.(D))

  0.082285 seconds (13.15 k allocations: 597.993 KiB)


LoadError: [91mBase.LinAlg.PosDefException(6)[39m

In [295]:
n = 10
A = randn(n,n)
A = A'A-1
isposdef(A)

false

In [299]:
A1 = makePSD_current(A); println(norm(A1-A))
A2 = makePSD_cur_mod(A); println(norm(A2-A))
A3 = makePSD_33(A); println(norm(A3-A))
A4 = makePSD_33_mod(A); println(norm(A4-A))
@test isposdef(A1)
@test isposdef(A2)
@test isposdef(A3)
@test isposdef(A4)


6.000000000000001
6.000000000000001
8.192000000000002
6.000000000000001


Test Passed

In [240]:
@btime makePSD_current(A)
@btime makePSD_cur_mod(A)
@btime makePSD_33(A)
@btime makePSD_33_mod(A)
@btime makePSD_eig(A)


  70.834 μs (153 allocations: 50.06 KiB)
  3.319 μs (21 allocations: 11.56 KiB)
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
it

iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
i

iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
i

iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
i

iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
iter: 14
i

...

iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4


iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4


iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4


iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4
iter: 4


10×10 Array{Float64,2}:
 11.4838   -1.52827   -2.92268    -1.79826   …   2.35048  -2.03012   -1.57598
 -1.52827   4.44252   -0.39112    -5.09409       4.40545  -2.1517     3.93476
 -2.92268  -0.39112    6.66942    -0.934389     -4.2691    3.2371    -5.80807
 -1.79826  -5.09409   -0.934389   12.4806       -9.34197  -0.152383  -3.57146
  7.17053   0.33979    0.538997   -3.22285       1.20422  -3.87027   -2.64133
 -4.02846  -0.452091   0.0652162  -1.47275   …   1.72738   3.5233     2.52712
 -2.37454   0.621814  -1.7431      0.124251      0.21401  -6.28214    2.3334 
  2.35048   4.40545   -4.2691     -9.34197      13.1073    1.70322    7.00123
 -2.03012  -2.1517     3.2371     -0.152383      1.70322  24.1696    -5.48382
 -1.57598   3.93476   -5.80807    -3.57146       7.00123  -5.48382   10.5315 

In [300]:
@btime makePSD_cur_mod(A)
@btime makePSD_33_mod(A)

  3.324 μs (29 allocations: 11.69 KiB)
  3.876 μs (24 allocations: 11.78 KiB)


10×10 Array{Float64,2}:
 11.7737     2.89585    0.92676   …  -0.525927   2.13677   -0.404215
  2.89585   17.456     -0.567525     -0.119875   0.863267   1.77378 
  0.92676   -0.567525  23.9885       -4.36711    2.92744   -1.18217 
  0.768197  -1.83128   -0.34562      -0.749195  -1.35473   -1.37446 
 -2.87009   -6.89811   -2.72296      -3.2579    -0.301738  -2.79509 
  1.02581   -0.189142   2.85542   …  -1.14569    0.857933  -1.15899 
 -1.66682   -1.60707    2.96481      -2.3955     0.897174  -2.0129  
 -0.525927  -0.119875  -4.36711       8.35986   -1.92542   -4.0816  
  2.13677    0.863267   2.92744      -1.92542   11.2871    -6.01697 
 -0.404215   1.77378   -1.18217      -4.0816    -6.01697   23.039   

In [304]:
t = @elapsed A2 = makePSD_33_mod(A)

6.6927e-5

# Regularization

In [47]:
QR = qrfact([S;eye(3)])[:R]


3×3 Array{Float64,2}:
 -10.0499    0.0      0.0   
   0.0     -20.025    0.0   
   0.0       0.0    -30.0167

In [44]:
qrfact(eye(3))[:Q]

3×3 Base.LinAlg.QRCompactWYQ{Float64,Array{Float64,2}}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [147]:
~isposdef(A)
A

30×30 Array{Float64,2}:
 0.566463   0.408761  0.412705   …  0.259195   0.327324   0.799589 
 0.41266    0.715937  0.685025      0.545495   0.67685    0.772635 
 0.776262   0.359029  0.962435      0.26185    0.477129   0.0585107
 0.324467   0.340059  0.427098      0.123077   0.449419   0.869349 
 0.21333    0.476333  0.531201      0.309509   0.975433   0.200981 
 0.488193   0.357906  0.427206   …  0.581146   0.66318    0.527108 
 0.0188466  0.026732  0.0591649     0.698513   0.232726   0.699423 
 0.462016   0.310867  0.254808      0.197934   0.130152   0.343449 
 0.0786047  0.141037  0.034218      0.526898   0.80801    0.623167 
 0.168239   0.937422  0.477134      0.382063   0.43268    0.948488 
 0.938757   0.98496   0.973969   …  0.268285   0.467763   0.644238 
 0.648538   0.80625   0.585099      0.477384   0.860873   0.782147 
 0.900065   0.340182  0.772688      0.5454     0.206425   0.0877941
 ⋮                               ⋱                                 
 0.13914    0.862709  0.

In [166]:


function roundPSD(A)
    return isposdef(round.(A,3))
end

# This is faster and more reliable
function convertPSD(A)
    return isposdef(Hermitian(A))
end
    

convertPSD (generic function with 1 method)

In [228]:
A = rand(10,10)
A = A'A
# A += randn(100,100)/1e10
Q = chol(A)
@time all(diag(Q.>0));
@time all(eigvals(Q).>0)
@btime all(x->x>0,diag($Q))

function checkdiag(A::AbstractArray)
    for i = 1:size(A,1)
        if A[i,i] < 0
            return false
        end
    end
    return true
end

@btime checkdiag($Q)

  0.049281 seconds (8.66 k allocations: 440.492 KiB)
  0.036567 seconds (6.96 k allocations: 371.485 KiB)
  82.122 ns (3 allocations: 224 bytes)
  13.927 ns (0 allocations: 0 bytes)


true

In [230]:
eigvals(A)

10-element Array{Float64,1}:
  0.00577512
  0.030949  
  0.0939108 
  0.164971  
  0.299013  
  0.620625  
  0.781933  
  1.69146   
  2.38737   
 22.8026    

In [193]:
@btime roundPSD(A)
@btime convertPSD(A);

  721.459 μs (30 allocations: 157.55 KiB)
  62.947 μs (5 allocations: 78.28 KiB)


In [233]:
eigvals(chol(A))

10-element Array{Float64,1}:
 1.04681 
 1.13689 
 1.45373 
 0.908388
 0.686168
 0.342361
 0.523021
 0.341086
 0.330954
 0.279052

In [234]:
diag(chol(A))

10-element Array{Float64,1}:
 1.04681 
 1.13689 
 1.45373 
 0.908388
 0.686168
 0.342361
 0.523021
 0.341086
 0.330954
 0.279052